In [1]:
# Libraries to install
# pip install pystan==2.19.1.1
# pip install prophet==1.0

# From Prophet to TorchProphet - A guide

Prophet is a popular human-in-the-loop forecasting framework build on Stan that allows to create forecasts and visualisations rapidly.

NeuralProphet is an evolution of Prophet that builds on PyTorch and supports even more forecasting components.

Since the interface of Prophet and NeuralProphet differ in various details, we build a wrapper for NeuralProphet, TorchProphet, that is 1:1 compatible with many Prophet applications.

## Disclaimer
Since TorchProphet is a wrapper for NeuralProphet, it should mainly be used for experiments and we do not encourage the use in production. It allows forecasting practitioners using Prophet to quickly evaluate the performance of both frameworks side by side.

## Outline
This tutorial will walk through building a simple forecasting model showing Prophet and TorchProphet code side-by-side. The code for Prophet comes from the official [Prophet quick-start tutorial](https://facebook.github.io/prophet/docs/quick_start.html#python-api).


### Data Loading

We will use the time series of the log daily page views for the Wikipedia page for Peyton Manning as example for this tutorial. 

In [2]:
import pandas as pd

data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
df = pd.read_csv(data_location + "wp_log_peyton_manning.csv")

### Model initialisation

This is the only line of code for most applications that needs to be changed for the migration using the wrapper.

In [ ]:
from prophet import Prophet
p = Prophet()

from neuralprophet import TorchProphet
np = TorchProphet()

### Fit forecasting models

In [ ]:
p.fit(df)

In [ ]:
np.fit(df)

### Expand the dataset into the future

In [ ]:
future_p = p.make_future_dataframe(periods=365)
future_p

In [ ]:
future_np = np.make_future_dataframe(periods=365)
future_np

### Predict future values

In [ ]:
forecast_p = p.predict(future_p)
forecast_p

In [ ]:
forecast_np = np.predict(future_np)
forecast_np

### Plot the forecasts

In [ ]:
fig1_p = p.plot(forecast_p)

In [ ]:
fig1_np = np.plot(forecast_np)

### Plot the forecast components

In [ ]:
fig2_p = p.plot_components(forecast_p)

In [ ]:
fig2_np = np.plot_components(forecast_np)

### Plot using Ploty

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(p, forecast_p)

In [ ]:
np.plot(forecast_np, plotting_backend="plotly")

## What's different between Prophet and TorchProphet?
Both the Prophet and the NeuralProphet model share the concept of decomposing a time series into it's components which allows a human to inspect and interprete the individual components of the forecast. Since NeuralProphet differs both in the functionality and the interface, the TorchProphet allows users to rapidly explore NeuralProphet's capabilities. In the following we will walk through the differences and similarities when working with TorchProphet instead of Prophet.

### Identical behaviour
- **Trend**  
  Both Prophet and TorchProphet provide a trend component that can be predicted using `predict_trend(...)` and configured using the `changepoints` or `n_changepoints` argument during initialization.
- **Seasonality**  
  Both tools allow to model seasonality by using the `add_seasonality(...)` function or providing the `yearly_seasonality`, `weekly_seasonality`, `daily_seasonality` or `seasonality_mode` argument during initialization.
- **Country holidays**  
  Both tools allow to add country holidays as special events to the forecast. This is archieved by using the `add_country_holidays(...)` function.
- **Future regressors**  
  Regressors can be added using the `add_regressor(...)` function in both tools.
  > **Info:** In TorchProphet, we differentiate between lagged and future regressors. The `add_regressor(...)` function of Prophet corresponds to the `add_future_regressor(...)` function of TorchProphet.
- **Holidays**  
  In Prophet, additional events to the country holidays can be added during initialization by passing an `holidays` argument. In TorchProphet, this can be archieved by passing the holidays to the `add_events(...)` function. TorchProphet supports both methods.

### Different behaviour
- **Regularization**  
  In Prophet, the argument `prior_scale` can be used to configure regularization. In TorchProphet, regularization is controlled via the `regularization` argument. `prior_scale` and `regularization` have an inverse relationship but cannot directly be translated. Thus, TorchProphet does not support regularization via `prior_scale`. Regularization is archieved via the `_reg` argument.
  ```python
  # Prophet
  from prophet import Prophet
  m = Prophet(seasonality_prior_scale=0.5)
  # Wrapper
  from neuralprophet import TorchProphet as Prophet
  m = Prophet(seasonality_reg=0.5)
  ```
- **Uncertainty**  
  In Prophet, all forecasts are configured to use uncertainty intervals automatically. In NeuralProphet, the uncertaintly intervals need to be configured by the user. TorchProphet uses the default uncertainty intervals as defined in Prophet to mirror the behviour.

  However, the uncertainty interval calculation differs between Prophet and NeuralProphet. Since Prophet uses a MAP estimate for uncertainty by default [Thread](https://github.com/facebook/prophet/issues/1124), NeuralProphet relies on quantile regression. Thus, the values are not directly comparable.
  ```python
  # Prophet
  from prophet import Prophet
  m = Prophet(interval_width=0.8)
  # Wrapper
  from neuralprophet import TorchProphet as Prophet
  m = Prophet(interval_width=[0.1, 0.9])
  ```


### Unsupported features in TorchProphet
- **Saturating forecasts**  
  Saturating forecasts limit the predicted value to a certain limit called capacity. In Prophet, this is archieved by passing the `growth='logistic'` argument during initialization and providing a capacity limit. This functionality is currently not supported by NeuralProphet.
- **Conditional seasonality**  
  Conditional seasonality allows to model certain events as seasonal effects (eg. whether it's currently NFL season). This can be archieved in Prophet by passing the argument `condition_name` to `add_seasonality(...)`. This feature is currently also not supported in NeuralProphet.

### Additional features of TorchProphet
- **Autoregression**  
  TorchProphet allows to model autoregression of arbitrary lag lengths by building on a Neural Network implementation of the autoregression algorithm (called AR-Net). More information can be found here [Autoregression](https://neuralprophet.com/html/autoregression_yosemite_temps.html).
- **Lagged regressors**  
  TorchProphet does not only support "future" regressors like in Prophet, but also lagged regressors that are simultaneous to the time series to predict. More information can be found here [Lagged covariates](https://neuralprophet.com/html/lagged_covariates_energy_ercot.html).
- **Global model**  
  TorchProphet supports hierachical forecasts by training a global model on many simultaneous time series that is used to improve the performance of predicting an individual time series. More infos can be found here [Global Model](https://neuralprophet.com/html/global_modeling.html).
- **Neural Network architecture**  
  TorchProphet models the forecast components using a Neural Network. By default, the network has only one hidden layer. However, for more complex time series, the depth of the network can be increased to learn more complex relations.
- **PyTorch**  
  TorchProphet is build on Pytorch (soon PyTorch Lightning) and thus provides interfaces for developers to extend the vanilla model for specific use cases.

### Prophet example notebook support
The following table provides an overview on the notebooks by Prophet that can be run without any further changes than changing the import of the forecasting tool from `from prophet import Prophet` to `from neuralprophet import TorchProphet as Prophet`.

| Prophet notebook  | Status  |  Comment  |
| ------------- | ------------- | ------------- |
| `additional_topics.ipynb`  | Not supported  | Additional features, specific to Prophet |
| `diagnostics.ipynb`  | Not supported  | Exploration of model components, specific to Prophet |
| `multiplicative_seasonality.ipynb`  | Supported  |  |
| `non-daily_data.ipynb`  | Supported  | Only minor plots are not supported |
| `outliers.ipynb`  | Supported  |  |
| `quick_start.ipynb`  | Supported  |  |
| `saturating_forecasts.ipynb`  | Not supported  | Saturating forecasts are not supported by NeuralProphet |
| `seasonality,_holiday_effects,_and_regressors.ipynb`  | Partly supported  | Events and holidays supported, Conditional seasonality not supported  |
| `trend_changepoints.ipynb`  | Supported  | Functionality supported, plots partly not supported |
| `uncertainty_intervals.ipynb`  | Supported  | Uncertainty intervals are supported, Bayesian sampling not |